## 用 Pandas 读入内嵌的数组

In [79]:
import pandas as pd, h5py, numpy as np

t_info = pd.read_hdf("1.in.h5", "TriggerInfo")
g_truth = pd.read_hdf("1.in.h5", "GroundTruth")
with h5py.File("1.in.h5") as ipt:
    wf = ipt['Waveform'][()]
wf.dtype

dtype([('EventID', '<i8'), ('ChannelID', '<i2'), ('Waveform', '<i2', (1029,))])

In [112]:
zdf = pd.DataFrame({"EventID":wf['EventID'], "ChannelID":wf["ChannelID"]})
zdf['Waveform'] = list(wf["Waveform"])
zdf.head()

,EventID,ChannelID,Waveform
0,1,0,"[971, 972, 971, 972, 972, 972, 975, 973, 971, ..."
1,1,1,"[971, 972, 973, 973, 973, 971, 973, 974, 970, ..."
2,1,2,"[972, 972, 973, 973, 973, 972, 971, 972, 975, ..."
3,1,4,"[973, 971, 972, 972, 970, 972, 971, 974, 972, ..."
4,1,5,"[975, 973, 973, 974, 974, 973, 973, 973, 971, ..."


## 数组被转化成了最一般的 object

In [20]:
zdf['Waveform'].dtype
# O 是 "Object" 之意

dtype('O')

## 用 Pandas 输出内嵌的数组

- 默认是 Object，需要对它进行转换

In [113]:
zo = zdf.to_records(index=False)
zo.dtype

dtype((numpy.record, [('EventID', '<i8'), ('ChannelID', '<i2'), ('Waveform', 'O')]))

In [115]:
type(zo['Waveform'][0])

numpy.ndarray

In [116]:
zs = np.array(list(zo['Waveform']), dtype=[("Waveform", np.int16, 1029)])
zs.dtype, zs.shape

(dtype([('Waveform', '<i2', (1029,))]), (296, 1029))

In [117]:
zt=np.zeros(len(zo), dtype=wf.dtype)
zt['EventID'] = zo['EventID']
zt['ChannelID'] = zo['ChannelID']
zt['Waveform'] = zs

In [118]:
with h5py.File("output.h5", 'w') as out:
    out.create_dataset("zt", data=zt, compression="gzip", shuffle=True)
    
# 输出了文件 `output.h5`

## Pandas 与关系代数的历史

- 关系代数最典型的实现是关系数据库 (Relational Database)，一般使用 SQL 语言描述
  - Oracle（甲骨文）公司数据库
  - MariaDB (MySQL 的后继)
  - PostgreSQL
  - SQLite
- 另一个分支是统计软件，比如 S 语言中的 SAS/SPSS 中的 Dataset 或 DataFrame
  - GNU R 以及对应的 R 语言，是一个 S 语言的后继，其中 DataFrame 是语言的核心
  - Pandas 受 GNU R 的影响，目标是在 Python 的语言环境中实现 DataFrame 及其基本操作
- MapReduce 分布式大数据算法也受到关系代数的影响，一般都会使用关系代数作为平台高级接口
  - Hadoop 生态圈, Spark, etc.

## 表格的约定
- 列代表参数
  例如：学号，班级，姓名
- 行代表样本
  例如：第一位同学，第二位

## 关系代数中的集合运算

- 集合运算：
  - 交、并、差，笛卡尔积
  - 常见的例子

## 特有运算：投影
- 即选择某一列

In [73]:
wf['EventID']

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8

## 特有运算：选择
- 选择符合要求的行

In [75]:
zdf.query("ChannelID==1")

,EventID,ChannelID,Waveform
1,1,1,"(971, 972, 973, 973, 973, 971, 973, 974, 970, ..."
27,2,1,"(973, 973, 972, 972, 972, 974, 973, 972, 971, ..."
57,3,1,"(972, 973, 971, 972, 972, 972, 971, 972, 974, ..."
87,4,1,"(972, 974, 972, 973, 972, 971, 973, 971, 971, ..."
117,5,1,"(973, 971, 974, 971, 971, 971, 972, 971, 975, ..."
147,6,1,"(971, 972, 972, 972, 974, 974, 972, 970, 971, ..."
177,7,1,"(974, 971, 972, 972, 973, 972, 971, 974, 971, ..."
207,8,1,"(972, 973, 972, 972, 972, 972, 973, 971, 971, ..."
237,9,1,"(974, 973, 972, 973, 972, 973, 971, 972, 972, ..."
267,10,1,"(973, 972, 974, 972, 972, 973, 973, 973, 972, ..."


## 特有运算：连接(join)
- 把两个关系（两张表），按照共同信息，组合成一个关系

In [81]:
pd.merge(t_info, g_truth, on="EventID").head()

,EventID,Sec,NanoSec,ChannelID,PETime
0,1,0,282050417,0,284
1,1,0,282050417,0,296
2,1,0,282050417,0,302
3,1,0,282050417,0,303
4,1,0,282050417,0,346


## 连接的细节区分
- 当公共列不完全一致时，可以：
  - 只保留公共部分：内连接
  - 左边全保留，右侧补 NULL：左连接
  - 与之对称的：右连接
  - 左右两侧都在必要时补 NULL：外连接

In [93]:
e1 = g_truth.query("EventID==1")
e2 = g_truth.query("EventID==2")
e2.head()

,EventID,ChannelID,PETime
311,2,0,295
312,2,0,311
313,2,0,320
314,2,0,325
315,2,0,332


In [97]:
pd.merge(e1, e2, on="ChannelID", how="left").head(10)
# 没有太大区别，例子不够明显

,EventID_x,ChannelID,PETime_x,EventID_y,PETime_y
0,1,0,284,2,295
1,1,0,284,2,311
2,1,0,284,2,320
3,1,0,284,2,325
4,1,0,284,2,332
5,1,0,296,2,295
6,1,0,296,2,311
7,1,0,296,2,320
8,1,0,296,2,325
9,1,0,296,2,332


## 特有运算 除法
- 笛卡尔积的逆运算，不作要求

## 关系代数之外的常用运算：GroupBy
- GroupBy 把表格按照某些标号分类，分别进行计算

In [98]:
zdf.groupby("EventID").count()

,ChannelID,Waveform
EventID,,
1,26,26
2,30,30
3,30,30
4,30,30
5,30,30
6,30,30
7,30,30
8,30,30
9,30,30


In [104]:
num_PE = g_truth.groupby(["EventID", "ChannelID"]).count()
num_PE.head()

PETime
EventID ChannelID        
1       0               5
        1               6
        2               4
        4               6
        5              49

In [108]:
avg_time = g_truth.groupby(["EventID", "ChannelID"]).mean()
avg_time.head()

PETime
EventID ChannelID            
1       0          306.200000
        1          316.500000
        2          302.000000
        4          302.666667
        5          303.673469

In [110]:
min_time = g_truth.groupby(["EventID", "ChannelID"]).min()
min_time.head()

PETime
EventID ChannelID        
1       0             284
        1             285
        2             284
        4             291
        5             268

In [111]:
avg_min = min_time.groupby("ChannelID").mean()  # average of the first hits
avg_min.head()

,PETime
ChannelID,
0,291.800000
1,290.600000
2,288.400000
3,289.666667
4,289.900000


## 关系代数之外的常用运算：SortBy
- 按照某些列排序

## 标准接口
- 这些基本操作覆盖了所有的数据处理场景
- 极大促进了社会分工，极大促进了人类处理数据的能力